In [1]:
import datajoint as dj
import numpy as np
import calcification_Module as cm
import os
import pathlib
import pymeshfix

In [2]:
#connect to the database and pull down Cleansed Mesh ones and generate the resurfacing of these
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")
schema = dj.schema("microns_ta3p100")



Connecting celiib@10.28.0.34:3306


In [3]:
ta3p100.Mesh()

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices)
1,648518346341351441,28781,57162,=BLOB=,=BLOB=
1,648518346341351458,17290,34404,=BLOB=,=BLOB=
1,648518346341351467,28754,57350,=BLOB=,=BLOB=
1,648518346341351476,5377,10720,=BLOB=,=BLOB=
1,648518346341351491,65512,130474,=BLOB=,=BLOB=
1,648518346341351495,4826,9640,=BLOB=,=BLOB=
1,648518346341351496,14848,29612,=BLOB=,=BLOB=


In [4]:
#function that will convert skeleton into numpy array to be stored
ta3p100.Mesh.describe()

# Pinky100 Mesh structures
-> ta3p100.Segment
---
n_vertices           : bigint                       # number of vertices in this mesh
n_triangles          : bigint                       # number of triangles in this mesh
vertices             : longblob                     # x,y,z coordinates of vertices
triangles            : longblob                     # triangles (triplets of vertices)



'# Pinky100 Mesh structures\n-> ta3p100.Segment\n---\nn_vertices           : bigint                       # number of vertices in this mesh\nn_triangles          : bigint                       # number of triangles in this mesh\nvertices             : longblob                     # x,y,z coordinates of vertices\ntriangles            : longblob                     # triangles (triplets of vertices)\n'

In [12]:
#figure out what keysource want
ta3p100.Neurite() & "neurite_type='axon'"  #32207 tuples

ta3p100.Mesh & ta3p100.Neurite  #& "decimation_ratio='0.35'"


segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices)
2,648518346341351467,28754,57350,=BLOB=,=BLOB=
2,648518346341351503,11192,22282,=BLOB=,=BLOB=
2,648518346341351508,58449,116104,=BLOB=,=BLOB=
2,648518346341351509,24919,49578,=BLOB=,=BLOB=
2,648518346341351512,37179,74128,=BLOB=,=BLOB=
2,648518346341351514,37788,75374,=BLOB=,=BLOB=
2,648518346341351518,20204,40278,=BLOB=,=BLOB=


In [13]:
ta3p100.Decimation35()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices,n_triangles,vertices,triangles
2,648518346341366885,0.35,338990,673206,=BLOB=,=BLOB=
2,648518346341371119,0.35,234848,460727,=BLOB=,=BLOB=
2,648518346341373204,0.35,2267151,4510325,=BLOB=,=BLOB=
2,648518346341380687,0.35,533632,1063782,=BLOB=,=BLOB=
2,648518346341382424,0.35,176358,351867,=BLOB=,=BLOB=
2,648518346341388647,0.35,2792025,5523545,=BLOB=,=BLOB=
2,648518346341388820,0.35,185381,364564,=BLOB=,=BLOB=


In [14]:
ta3p100.DecimationOrphan()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices,n_triangles,vertices,triangles
2,648518346341352006,0.35,32107,64022,=BLOB=,=BLOB=
2,648518346341352223,0.35,39145,77867,=BLOB=,=BLOB=
2,648518346341353019,0.35,8809,17594,=BLOB=,=BLOB=
2,648518346341353186,0.35,44780,88967,=BLOB=,=BLOB=
2,648518346341353521,0.35,7325,14339,=BLOB=,=BLOB=
2,648518346341353574,0.35,59341,117923,=BLOB=,=BLOB=
2,648518346341353607,0.35,68762,136431,=BLOB=,=BLOB=


In [19]:
# 702 tuples  +  396 tuples = 1098 tuples
ta3p100.DecimationOrphan.proj() + ta3p100.Decimation35.proj()
# 702 tuples  +  396 tuples + 32207 = 33305 tuples
total_contacts_segment = (dj.U("segmentation","segment_id") & (ta3p100.DecimationOrphan.proj() + ta3p100.Decimation35.proj())) + (ta3p100.Mesh & ta3p100.Neurite).proj()

#seem to be missing about 15

In [23]:
total_contacts_segment * ta3p100.DecimationOrphan 

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices,n_triangles,vertices,triangles
2,648518346341352006,0.35,32107,64022,=BLOB=,=BLOB=
2,648518346341352223,0.35,39145,77867,=BLOB=,=BLOB=
2,648518346341353019,0.35,8809,17594,=BLOB=,=BLOB=
2,648518346341353186,0.35,44780,88967,=BLOB=,=BLOB=
2,648518346341353521,0.35,7325,14339,=BLOB=,=BLOB=
2,648518346341353574,0.35,59341,117923,=BLOB=,=BLOB=
2,648518346341353607,0.35,68762,136431,=BLOB=,=BLOB=


In [ ]:
648518346341400866

In [ ]:
@schema
class SkeletonPymesh(dj.Computed):
    definition = """
    -> ta3p100.Mesh
    ---
    n_edges   : number of edges stored
    edges     : array storing vertices of edges and each seperated by Nan

    """
    
    key_source = (((dj.U("segmentation","segment_id") & 
                  (ta3p100.DecimationOrphan.proj() + 
                   ta3p100.Decimation35.proj())) + 
                  (ta3p100.Mesh & ta3p100.Neurite).proj()) & "segmentation=2")
    
    def meke(self,key):
        ##Get the vertices and triangle data
        #1) check to see if neuron is in regular exhitatory neuron table
        #Set the error label flags for later so can filter away error labels
        
        #2) if not check if in the orphan table
        #Set the error label flags for later so can filter away error labels
        
        #if not use the mesh table (no error label flag set)
        
        #if error flag is set filter away those label
